In [ ]:

%load_ext autoreload
%autoreload 2

import jax.numpy as jnp
import jax
import matplotlib.pyplot as plt
import pymudokon as pm
import optax

jax.config.update('jax_platform_name', 'cpu')
# jax.config.update("jax_enable_x64", True)
import pandas as pd

from jax.scipy.optimize import minimize


In [ ]:
# d = 1, rhop = 1, e = 0.7, mu = 0, K = 10^3, shear rate = 1
names = ["nu", "p", "s", "T", "I", "CN", "k"]
df_k3 = pd.read_table("../DATA_SS/k1ex.data",sep=" ", skiprows=2,nrows=21, names = names)
df_k4 = pd.read_table("../DATA_SS/k1ex.data",sep=" ", skiprows=26,nrows=21, names = names)
df_k5 = pd.read_table("../DATA_SS/k1ex.data",sep=" ", skiprows=50,nrows=21, names = names)
df_k6 = pd.read_table("../DATA_SS/k1ex.data",sep=" ", skiprows=74,nrows=26, names= names)
df_k7 = pd.read_table("../DATA_SS/k1ex.data",sep=" ", skiprows=103,nrows=21, names= names)
df_k8 = pd.read_table("../DATA_SS/k1ex.data",sep=" ", skiprows=127,nrows=21, names= names)

In [ ]:
def df_format(df,d,K):
    df["pstar"] = df["p"]*(d/K)
    df["mu"] = df["s"]/df["p"]

    df["e"] = 1./df["nu"] -1
    return df



df_k3 = df_format(df_k3,d=1,K=10**3)
df_k4 = df_format(df_k4,d=1,K=10**4)
df_k5 = df_format(df_k5,d=1,K=10**5)
df_k6 = df_format(df_k6,d=1,K=10**6)
df_k7 = df_format(df_k7,d=1,K=10**7)
df_k8 = df_format(df_k8,d=1,K=10**8)

In [ ]:
# import scipy

# import scipy.optimize


# phi_c =0.634
# p_phi=0.9
# I_phi= 3.28
# rho_p=1
# dgamma_dt=1
# d=1
# def dilatancy(p):
    
#     term1 = I_phi*jnp.log(phi_c / phi)
#     term2 = (dgamma_dt*d)/jnp.sqrt(p/rho_p)
#     sol = term1 - term2
#     return sol[0]


# tol = jnp.array([1.0])
# fig, ax = plt.subplots()
# phi_stack = jnp.arange(0.5,0.7,0.01)
# p_stack = []
# for phi in phi_stack:
#     sol = minimize(dilatancy, tol,method="BFGS")
#     # a = jax.lax.custom_root(dilatancy, tol)   
#     # sol = scipy.optimize.fsolve(dilatancy, tol, args=(phi, phi_c, I_phi, dgamma_dt, rho_p,d))
#     p_stack.append(sol.x[0])
#     # sol = scipy.optimize.minimize(dilatancy, tol, args=(phi, phi_c, I_phi, dgamma_dt, rho_p,d))
#     # p_stack.append(sol.x[0])
    

# phi_stack = np.array(phi_stack)
# # ax.plot(phi_stack, p_stack)
# ax.plot(phi_stack, jnp.nan_to_num(p_stack), ls="--",lw=2, label="Impl")
# df_k8.plot(y="p", x="nu", kind="scatter", ax = ax, label="K = 10^8")
# ax.set_yscale("log")


# # ax.set_xlim(0.5,0.7)

# # ax.legend()

In [ ]:
import optimistix as optx
import numpy as np

def fn(p,args):
    phi = args
    phi_c =0.634
    p_phi=0.33
    I_phi= 3.28
    rho_p=1
    dgamma_dt=1
    d=1
    k=10**3
    
    lam = 0.1
    term1 = lam*jnp.log(p) +  I_phi*jnp.log(phi_c / phi)
    # term1 = p*(d/k)/p_phi + I_phi*jnp.log(phi_c / phi)
    term2 = (dgamma_dt*d)/jnp.sqrt(p/rho_p)
    sol = term1 - term2
    return sol

solver = optx.Newton(rtol=1e-3,atol=1e-8)

# y0 = jnp.array([.3])

fig, ax = plt.subplots()
phi_stack = jnp.arange(0.3,0.7,0.01)
p_stack = []
for phi in phi_stack:
    sol = optx.root_find(fn,solver,0.3,args=phi,throw=False)

    p_stack.append(sol.value)

    

phi_stack = np.array(phi_stack)
# ax.plot(phi_stack, p_stack)
ax.plot(phi_stack, np.nan_to_num(p_stack)/10**3, ls="--",lw=2, label="Impl")
df_k8.plot(y="pstar", x="nu", kind="scatter", ax = ax, label="K = 10^8")
df_k3.plot(y="pstar", x="nu", kind="scatter", ax = ax, label="K = 10^3", color="red")
ax.set_yscale("log")
# ax.set_xlim(0.5,0.7)
# ax.set_ylim(0,10e7)

In [ ]:
sol